---
title: Quantum circuit optimization
description: This lesson will address several aspects of circuit optimization in quantum computing.
---


 # Quantum circuit optimization

<Admonition type="note">

Toshinari Itoko (21 June 2024)

[Download the pdf](https://ibm.ent.box.com/public/static/0hvvgr1gnwx64x2ukgk04sss6sxc4zko.zip) of the original lecture. Note that some code snippets might become deprecated since these are static images.

*Approximate QPU time to run this experiment is 15 s.*

(Note: Some cells of part 2 are copied from the notebook "Qiskit Deep dive", written by Matthew Treinish (Qiskit maintainer))

</Admonition>

In [1]:
# !pip install 'qiskit[visualization]'
# !pip install qiskit_ibm_runtime qiskit_aer
# !pip install jupyter
# !pip install matplotlib pylatexenc pydot pillow

In [1]:
import qiskit

qiskit.__version__

'2.0.2'

In [2]:
import qiskit_ibm_runtime

qiskit_ibm_runtime.__version__

'0.40.1'

In [3]:
import qiskit_aer

qiskit_aer.__version__

'0.17.1'

## 1. Introduction

This lesson will address several aspects of circuit optimization in quantum computing. Specifically, we will see the value of circuit optimization by using optimization settings built into Qiskit. Then we will go a bit deeper and see what you can do as an expert in your particular application area to build circuits in a smart way. Finally, we will take a close look at what goes on during transpilation that helps us optimize our circuits.

## 2. Circuit optimization matters

We first compare the results of running 5-qubit GHZ state ($\frac{1}{\sqrt{2}} \left( |00000\rangle + |11111\rangle \right)$) preparation circuits with and without optimization.

In [4]:
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import BackendSamplerV2 as Sampler

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

backend = FakeBrisbane()

We first use a GHZ circuit naively synthesized as follows.

In [6]:
num_qubits = 5

ghz_circ = QuantumCircuit(num_qubits)
ghz_circ.h(0)
[ghz_circ.cx(0, i) for i in range(1, num_qubits)]
ghz_circ.measure_all()
ghz_circ.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/262b97e5-0.avif" alt="Output of the previous code cell" />

### 2.1 Optimization level
There are 4 available `optimization_level`s from 0-3. The higher the optimization level the more computational effort is spent to optimize the circuit. Level 0 performs no optimization and just does the minimal amount of work to make the circuit runnable on the selected backend. Level 3 spends the most amount if effort (and typically runtime) to try to optimize the circuit. Level 1 is the default optimization level.

We transpile the circuit without optimization (`optimization_level=0`) and with optimization (`optimization_level=2`).
We see a big difference in the circuit length of transpiled circuits.

In [7]:
pm0 = generate_preset_pass_manager(
    optimization_level=0, backend=backend, seed_transpiler=777
)
pm2 = generate_preset_pass_manager(
    optimization_level=2, backend=backend, seed_transpiler=777
)
circ0 = pm0.run(ghz_circ)
circ2 = pm2.run(ghz_circ)
print("optimization_level=0:")
display(circ0.draw("mpl", idle_wires=False, fold=-1))
print("optimization_level=2:")
display(circ2.draw("mpl", idle_wires=False, fold=-1))

optimization_level=0:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/042d2bbc-1.avif" alt="Output of the previous code cell" />

optimization_level=2:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/042d2bbc-3.avif" alt="Output of the previous code cell" />

### 2.2 Exercise
Try `optimization_level=1` as well and compare the resulting circuit with the above two. Try it by modifying the code above.

__Solution:__

In [8]:
pm1 = generate_preset_pass_manager(
    optimization_level=1, backend=backend, seed_transpiler=777
)
circ1 = pm1.run(ghz_circ)
print("optimization_level=1:")
display(circ1.draw("mpl", idle_wires=False, fold=-1))

optimization_level=1:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/6e8389e1-1.avif" alt="Output of the previous code cell" />

Run on a fake backend (noisy simulation). See Appendix 1 for how to run on a real backend.

In [9]:
# run the circuits on the fake backend (noisy simulator)
sampler = Sampler(backend=backend)
job = sampler.run([circ0, circ2], shots=10000)
print(f"Job ID: {job.job_id()}")

Job ID: 93a4ac70-e3ea-44ad-aea9-5045840c9076


In [10]:
# get results
result = job.result()
unoptimized_result = result[0].data.meas.get_counts()
optimized_result = result[1].data.meas.get_counts()

In [11]:
from qiskit.visualization import plot_histogram

# plot
sim_result = {"0" * 5: 0.5, "1" * 5: 0.5}
plot_histogram(
    [result for result in [sim_result, unoptimized_result, optimized_result]],
    bar_labels=False,
    legend=[
        "ideal",
        "no optimization",
        "with optimization",
    ],
)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/5d344bb9-0.avif" alt="Output of the previous code cell" />

## 3. Circuit synthesis matters

We next compare the results of running two differently synthesized 5-qubit GHZ state ($\frac{1}{\sqrt{2}} \left( |00000\rangle + |11111\rangle \right)$) preparation circuits.

In [12]:
# Original GHZ circuit (naive synthesis)
ghz_circ.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/896dc520-0.avif" alt="Output of the previous code cell" />

In [13]:
# A cleverly-synthesized GHZ circuit
ghz_circ2 = QuantumCircuit(5)
ghz_circ2.h(2)
ghz_circ2.cx(2, 1)
ghz_circ2.cx(2, 3)
ghz_circ2.cx(1, 0)
ghz_circ2.cx(3, 4)
ghz_circ2.measure_all()
ghz_circ2.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/d27a9d9b-0.avif" alt="Output of the previous code cell" />

In [14]:
# transpile both with the same optimization level 2
circ_org = pm2.run(ghz_circ)
circ_new = pm2.run(ghz_circ2)
print("original synthesis:")
display(circ_org.draw("mpl", idle_wires=False, fold=-1))
print("new synthesis:")
display(circ_new.draw("mpl", idle_wires=False, fold=-1))

original synthesis:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/d4e16053-1.avif" alt="Output of the previous code cell" />

new synthesis:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/d4e16053-3.avif" alt="Output of the previous code cell" />

The new synthesis produces a shallower circuit. Why?

This is because the new circuit can be laid out on linearly connected qubits, so on IBM&reg; Brisbane's heavy-hexagon coupling graph as well, while the original circuit requires star-shaped connectivity (a degree-4 node) and hence cannot be laid out on the heavy-hex coupling graph, which has nodes at most degree 3. As a result, the original circuit requires qubit routing that adds SWAP gates, increasing the gate count.

What we have done in the new circuit can be seen as a manual "coupling constraint-aware" circuit synthesis. In other words: manually solving circuit synthesis and circuit mapping at the same time.

In [15]:
# run the circuits
sampler = Sampler(backend=backend)
job = sampler.run([circ_org, circ_new], shots=10000)
print(f"Job ID: {job.job_id()}")

Job ID: 19d635b0-4d8b-44c2-a76e-49e4b9078b1b


In [16]:
# get results
result = job.result()
synthesis_org_result = result[0].data.meas.get_counts()
synthesis_new_result = result[1].data.meas.get_counts()

In [17]:
# plot
sim_result = {"0" * 5: 0.5, "1" * 5: 0.5}
plot_histogram(
    [
        result
        for result in [
            sim_result,
            unoptimized_result,
            synthesis_org_result,
            synthesis_new_result,
        ]
    ],
    bar_labels=False,
    legend=[
        "ideal",
        "no optimization",
        "synthesis_org",
        "synthesis_new",
    ],
)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/80f8ef81-0.avif" alt="Output of the previous code cell" />

In general, circuit synthesis depends on application and it's too difficult for a software to cover all possible applications. Qiskit transpiler happens to have no functions of synthesizing GHZ state preparation circuit. In such a case, manual circuit synthesis as shown above is worth considering.

In this section, we look into the details of how Qiskit transpiler works using the following toy example circuit.

In [ ]:
# Build a toy example circuit
from math import pi
import itertools
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import excitation_preserving

circuit = QuantumCircuit(4, name="Example circuit")
circuit.append(excitation_preserving(4, reps=1, flatten=True), range(4))
circuit.measure_all()

value_cycle = itertools.cycle([0, pi / 4, pi / 2, 3 * pi / 4, pi, 2 * pi])
circuit.assign_parameters(
    [x[1] for x in zip(range(len(circuit.parameters)), value_cycle)], inplace=True
)
circuit.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/f2228937-0.avif" alt="Output of the previous code cell" />

### 3.1 Draw the entire Qiskit transpilation flow

We look into the transpiler passes (tasks) for `optimization_level=1`.

In [19]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# There is no need to read this entire image, but this outputs all the steps in the transpile() call
# for optimization level 1
pm = generate_preset_pass_manager(1, backend, seed_transpiler=42)
pm.draw()

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/74bd20af-0.avif" alt="Output of the previous code cell" />

The flow consists of six stages:

In [20]:
print(pm.stages)

('init', 'layout', 'routing', 'translation', 'optimization', 'scheduling')


### 3.2 Draw an individual stage

First, let's draw all the tasks (transpiler passes) done in the `init` stage.

In [21]:
pm.init.draw()

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/09b4ffbe-0.avif" alt="Output of the previous code cell" />

We can run each individual stage. Let's run `init` stage for our circuit. By enabling logger, we can see the details of the run.

In [22]:
import logging

logger = logging.getLogger()
logger.setLevel("INFO")

init_out = pm.init.run(circuit)
init_out.draw("mpl", fold=-1)

INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.03576 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.16618 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.07176 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.27299 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: ContractIdleWiresInControlFlow - 0.00811 (ms)


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/a139da85-e5b4-4c7c-900f-da8a0b8a5989-1.avif" alt="Output of the previous code cell" />

### 3.3 Exercise
Draw `layout` stage passes and run the stage for the output circuit of the `init` stage (`init_out`), by modifying cells used above.

__Solution:__

In [23]:
display(pm.layout.draw())
layout_out = pm.layout.run(init_out)
layout_out.draw("mpl", idle_wires=False, fold=-1)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/56024db6-0.avif" alt="Output of the previous code cell" />

INFO:qiskit.passmanager.base_tasks:Pass: SetLayout - 0.01001 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: TrivialLayout - 0.07129 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: CheckMap - 0.08917 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: VF2Layout - 1.24431 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BarrierBeforeFinalMeasurements - 0.02599 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: SabreLayout - 5.11169 (ms)


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/56024db6-2.avif" alt="Output of the previous code cell" />

Do the same thing for `translation` stage.

__Solution:__

In [24]:
display(pm.translation.draw())
basis_out = pm.translation.run(layout_out)
basis_out.draw("mpl", idle_wires=False, fold=-1)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/fd7cec6b-0.avif" alt="Output of the previous code cell" />

INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.03386 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.02718 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 2.64192 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: CheckGateDirection - 0.02217 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: GateDirection - 0.36502 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.64778 (ms)


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/fd7cec6b-2.avif" alt="Output of the previous code cell" />

Note: Any individual stage cannot always be run independently (as some of them need to carry over information from one previous stage).

### 3.4 Optimization Stage

The last default stage in the pipeline is optimization. After we've embedded the circuit for the target the circuit has expanded quite a bit. Most of this is due to inefficiencies in the equivalence relationships from basis translation and swap insertion. The optimization stage is used to try and minimize the size and depth of the circuit. It runs a series of passes in a ```do while``` loop until it reaches a steady output.

In [25]:
# pm.pre_optimization.draw()
pm.optimization.draw()

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/f86b9045-0.avif" alt="Output of the previous code cell" />

In [27]:
logger = logging.getLogger()
logger.setLevel("INFO")
opt_out = pm.optimization.run(basis_out)

INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.30112 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.03195 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Size - 0.01216 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.01001 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Optimize1qGatesDecomposition - 0.63729 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.41723 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: ContractIdleWiresInControlFlow - 0.01192 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: GatesInBasis - 0.05484 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.08583 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.20599 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Size - 0.00787 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00715 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Optimize1qGatesDecomposition - 0.16809 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.17190 (ms)
INFO:qis

In [28]:
opt_out.draw("mpl", idle_wires=False, fold=-1)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/65d650b0-ec27-4b1b-a121-f1bb958b18e2-0.avif" alt="Output of the previous code cell" />

## 4. In-depth examples
### 4.1 Two-qubit block optimization using two-qubit unitary synthesis

For level 2 and 3, we have more passes (`Collect2qBlocks`, `ConsolidateBlocks`, `UnitarySynthesis`) for more optimization, namely two-qubit block optimization. (Compare the optimization stage flow for level 2 with that above for level 1)

The two-qubit block optimization is composed of two steps: Collecting and consolidating 2-qubit blocks and synthesizing the 2-qubit unitary matrices.

In [29]:
pm2 = generate_preset_pass_manager(2, backend, seed_transpiler=42)
pm2.optimization.draw()

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/179b1440-0.avif" alt="Output of the previous code cell" />

In [30]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    Collect2qBlocks,
    ConsolidateBlocks,
    UnitarySynthesis,
)

# Collect 2q blocks and consolidate to unitary when we expect that we can reduce the 2q gate count for that unitary
consolidate_pm = PassManager(
    [
        Collect2qBlocks(),
        ConsolidateBlocks(target=backend.target),
    ]
)

In [36]:
display(basis_out.draw("mpl", idle_wires=False, fold=-1))

consolidated = consolidate_pm.run(basis_out)
consolidated.draw("mpl", idle_wires=False, fold=-1)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/bbf4fa9a-6b49-4833-82fd-b3821f6bcb78-0.avif" alt="Output of the previous code cell" />

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/bbf4fa9a-6b49-4833-82fd-b3821f6bcb78-1.avif" alt="Output of the previous code cell" />

In [37]:
# Synthesize unitaries
UnitarySynthesis(target=backend.target)(consolidated).draw(
    "mpl", idle_wires=False, fold=-1
)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/7e7e0d3b-d267-4b1c-b207-42556d1ff3f2-0.avif" alt="Output of the previous code cell" />

In [38]:
logger.setLevel("WARNING")

We saw in Part 2 that the real quantum compiler flow is not that simple and is composed of many passes (tasks). This is mainly due to the software engineering required to ensure performance for a wide range of application circuits and maintainability of the software. Qiskit transpiler would work well in most cases but if you happen to see your circuit is not well optimized by Qiskit transpiler, it would be a good opportunity to research your own application-specific circuit optimization as shown in Part 1. Transpiler technology is evolving, your R&D contribution is welcome.

In [39]:
from qiskit.circuit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")
sampler = Sampler(backend)

In [43]:
circ = QuantumCircuit(3)
circ.ccx(0, 1, 2)
circ.measure_all()
circ.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/30a84ca1-0.avif" alt="Output of the previous code cell" />

In [44]:
sampler.run([circ])  # IBMInputValueError will be raised

IBMInputValueError: 'The instruction ccx on qubits (0, 1, 2) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (/docs/guides/transpile) for instructions to transform circuits and the primitive examples (/docs/guides/primitives-examples) to see this coupled with operator transformations.'

### 4.2 Circuit optimization matters

We first compare the results of running 5-qubit GHZ state ($\frac{1}{\sqrt{2}} \left( |00000\rangle + |11111\rangle \right)$) preparation circuits with and without optimization.

In [45]:
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

In [ ]:
service = QiskitRuntimeService()

In [ ]:
# backend = service.backend('ibm_brisbane')
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)  # Eagle
backend

We first use a GHZ circuit naively synthesized as follows.

In [55]:
num_qubits = 5

ghz_circ = QuantumCircuit(num_qubits)
ghz_circ.h(0)
[ghz_circ.cx(0, i) for i in range(1, num_qubits)]
ghz_circ.measure_all()
ghz_circ.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/485b8ce6-0.avif" alt="Output of the previous code cell" />

We transpile the circuit without optimization (`optimization_level=0`) and with optimization (`optimization_level=2`).
As you can see, there is a big difference in the circuit length of transpiled circuits.

In [56]:
pm0 = generate_preset_pass_manager(
    optimization_level=0, backend=backend, seed_transpiler=777
)
pm2 = generate_preset_pass_manager(
    optimization_level=2, backend=backend, seed_transpiler=777
)
circ0 = pm0.run(ghz_circ)
circ2 = pm2.run(ghz_circ)
print("optimization_level=0:")
display(circ0.draw("mpl", idle_wires=False, fold=-1))
print("optimization_level=2:")
display(circ2.draw("mpl", idle_wires=False, fold=-1))

optimization_level=0:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/87b861e2-1.avif" alt="Output of the previous code cell" />

optimization_level=2:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/87b861e2-3.avif" alt="Output of the previous code cell" />

In [57]:
# run the circuits
sampler = Sampler(backend)
job = sampler.run([circ0, circ2], shots=10000)
job_id = job.job_id()
print(f"Job ID: {job_id}")

Job ID: d13rnnemya70008ek1zg


In [58]:
# REPLACE WITH YOUR OWN JOB IDS
job = service.job(job_id)

In [59]:
# get results
result = job.result()
unoptimized_result = result[0].data.meas.get_counts()
optimized_result = result[1].data.meas.get_counts()

In [60]:
from qiskit.visualization import plot_histogram

# plot
sim_result = {"0" * 5: 0.5, "1" * 5: 0.5}
plot_histogram(
    [result for result in [sim_result, unoptimized_result, optimized_result]],
    bar_labels=False,
    legend=[
        "ideal",
        "no optimization",
        "with optimization",
    ],
)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/7527976e-0.avif" alt="Output of the previous code cell" />

### 4.3 Circuit synthesis matters

We next compare the results of running two differently synthesized 5-qubit GHZ state ($\frac{1}{\sqrt{2}} \left( |00000\rangle + |11111\rangle \right)$) preparation circuits.

In [61]:
# Original GHZ circuit (naive synthesis)
ghz_circ.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/886d9b45-0.avif" alt="Output of the previous code cell" />

In [62]:
# A better GHZ circuit (smarter synthesis), you learned in a previous lecture
ghz_circ2 = QuantumCircuit(5)
ghz_circ2.h(2)
ghz_circ2.cx(2, 1)
ghz_circ2.cx(2, 3)
ghz_circ2.cx(1, 0)
ghz_circ2.cx(3, 4)
ghz_circ2.measure_all()
ghz_circ2.draw("mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/3b559186-0.avif" alt="Output of the previous code cell" />

In [63]:
circ_org = pm2.run(ghz_circ)
circ_new = pm2.run(ghz_circ2)
print("original synthesis:")
display(circ_org.draw("mpl", idle_wires=False, fold=-1))
print("new synthesis:")
display(circ_new.draw("mpl", idle_wires=False, fold=-1))

original synthesis:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/054890b6-1.avif" alt="Output of the previous code cell" />

new synthesis:


<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/054890b6-3.avif" alt="Output of the previous code cell" />

In [64]:
# run the circuits
sampler = Sampler(backend)
job = sampler.run([circ_org, circ_new], shots=10000)
job_id = job.job_id()
print(f"Job ID: {job_id}")

Job ID: d13rp283grvg008j12fg


In [66]:
# REPLACE WITH YOUR OWN JOB IDS
job = service.job(job_id)

In [67]:
# get results
result = job.result()
synthesis_org_result = result[0].data.meas.get_counts()
synthesis_new_result = result[1].data.meas.get_counts()

In [68]:
# plot
sim_result = {"0" * 5: 0.5, "1" * 5: 0.5}
plot_histogram(
    [result for result in [sim_result, synthesis_org_result, synthesis_new_result]],
    bar_labels=False,
    legend=[
        "ideal",
        "synthesis_org",
        "synthesis_new",
    ],
)

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/b9021da5-0.avif" alt="Output of the previous code cell" />

### 4.4 General 1-qubit gate decomposition

In [69]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library.standard_gates import UGate

phi, theta, lam = Parameter("φ"), Parameter("θ"), Parameter("λ")

In [70]:
qc = QuantumCircuit(1)
qc.append(UGate(theta, phi, lam), [0])
qc.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/ed93f69a-0.avif" alt="Output of the previous code cell" />

In [42]:
transpile(qc, basis_gates=["rz", "sx"]).draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/2fa17bd2-0.avif" alt="Output of the previous code cell" />

### 4.5 One-qubit block optimization

In [71]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(1)
qc.x(0)
qc.y(0)
qc.z(0)
qc.rx(1.23, 0)
qc.ry(1.23, 0)
qc.rz(1.23, 0)
qc.h(0)
qc.s(0)
qc.t(0)
qc.sx(0)
qc.sdg(0)
qc.tdg(0)
qc.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/6f64d07d-0.avif" alt="Output of the previous code cell" />

In [72]:
from qiskit.quantum_info import Operator

Operator(qc)

Operator([[ 0.45292511-0.57266982j, -0.66852684-0.14135058j],
          [ 0.14135058+0.66852684j, -0.57266982+0.45292511j]],
         input_dims=(2,), output_dims=(2,))


In [73]:
from qiskit import transpile

qc_opt = transpile(qc, basis_gates=["rz", "sx"])
qc_opt.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/c06f5e75-0.avif" alt="Output of the previous code cell" />

In [74]:
Operator(qc_opt)

Operator([[ 0.45292511-0.57266982j, -0.66852684-0.14135058j],
          [ 0.14135058+0.66852684j, -0.57266982+0.45292511j]],
         input_dims=(2,), output_dims=(2,))


In [75]:
Operator(qc).equiv(Operator(qc_opt))

True

### 4.6 Toffoli decomposition

In [76]:
qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
qc.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/f802c5df-0.avif" alt="Output of the previous code cell" />

In [77]:
from qiskit import QuantumCircuit, transpile

qc = QuantumCircuit(3)
qc.ccx(0, 1, 2)
qc = transpile(qc, basis_gates=["rz", "sx", "cx"])
qc.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/330cea7e-0.avif" alt="Output of the previous code cell" />

### 4.7 CU gate decomposition

In [78]:
from qiskit.circuit.library.standard_gates import CUGate

phi, theta, lam, gamma = Parameter("φ"), Parameter("θ"), Parameter("λ"), Parameter("γ")
qc = QuantumCircuit(2)
# qc.cu(theta, phi, lam, gamma, 0, 1)
qc.append(CUGate(theta, phi, lam, gamma), [0, 1])
qc.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/1df5876d-0.avif" alt="Output of the previous code cell" />

In [79]:
from qiskit.circuit.library.standard_gates import CUGate

phi, theta, lam, gamma = Parameter("φ"), Parameter("θ"), Parameter("λ"), Parameter("γ")
qc = QuantumCircuit(2)
qc.append(CUGate(theta, phi, lam, gamma), [0, 1])
qc = transpile(qc, basis_gates=["rz", "sx", "cx"])
qc.draw(output="mpl")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/64f7e5f3-0.avif" alt="Output of the previous code cell" />

### 4.8 CX, ECR, CZ equal up to local Cliffords

Note that $H$(Hadamard), $S$($\pi/2$ Z-rotation), $S^\dagger$($-\pi/2$ Z-rotation), $X$(Pauli X) are all Clifford gates.

In [80]:
qc = QuantumCircuit(2)
qc.cx(0, 1)
qc.draw(output="mpl", style="bw")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/f5b362b6-0.avif" alt="Output of the previous code cell" />

In [81]:
qc = QuantumCircuit(2)
qc.cx(0, 1)
transpile(qc, basis_gates=["x", "s", "h", "sdg", "ecr"]).draw(output="mpl", style="bw")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/8740d07b-0.avif" alt="Output of the previous code cell" />

In [82]:
qc = QuantumCircuit(2)
qc.cx(0, 1)
transpile(qc, basis_gates=["h", "cz"]).draw(output="mpl", style="bw")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/5113a7c5-0.avif" alt="Output of the previous code cell" />

Using IBM backend 1q basis gates "rz", "sx" and "x".

In [83]:
qc = QuantumCircuit(2)
qc.cx(0, 1)
transpile(qc, basis_gates=["rz", "sx", "x", "ecr"]).draw(output="mpl", style="bw")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/9d9b54d4-0.avif" alt="Output of the previous code cell" />

In [84]:
qc = QuantumCircuit(2)
qc.cx(0, 1)
transpile(qc, basis_gates=["rz", "sx", "x", "cz"]).draw(output="mpl", style="bw")

<Image src="/learning/images/courses/utility-scale-quantum-computing/quantum-circuit-optimization/extracted-outputs/c395cd24-0.avif" alt="Output of the previous code cell" />

In [85]:
# Check Qiskit version
import qiskit

qiskit.__version__

'2.0.2'